In [11]:
# Atnaujintas variantas, po to, kaip padaryti pakeitimai failų varduose 2022 03 26
# reikia dar tvarkyti ///////////////////////////////////////////////////////////
# Nuskaito ir parodo anotacijas visiems failams iš sąrašo. Sąrašas sukuriamas su 0_zive_suppl_list

import pandas as pd
import numpy as np

import json, sys
from pathlib import Path
from icecream import ic

from zive_util_vu import get_annotations_distribution

import warnings
# warnings.filterwarnings("ignore")

my_os=sys.platform
print("OS in my system : ", my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Pasiruošimas

# //////////////// NURODOMI PARAMETRAI /////////////////////////////////////////////////////

# Bendras duomenų aplankas

if OS == 'Windows':
    Duomenu_aplankas = 'D:\DI\Data\MIT&ZIVE\VU'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI/Data/MIT&ZIVE/VU'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

# Vietinės talpyklos aplankas
db_folder = 'DUOM_VU'

# Aplankas su  EKG įrašais ir anotacijomis (.json)
rec_folder = 'records_selected'

# ///////////////////////////////////////////////////////////////////////////////////////////
print('\nSkriptas parodo anotacijų statistiką vietinėje ZIVE talpykloje')

pd.set_option("display.max_rows", 6000, "display.max_columns", 20)
pd.set_option('display.width', 1000)
# How to print an entire pandas DataFrame in Python
# https://www.kite.com/python/answers/how-to-print-an-entire-pandas-dataframe-in-python

# Nuoroda į aplanką su vietiniu EKG duomenų rinkiniu
db_path = Path(Duomenu_aplankas, db_folder, rec_folder) # vietinė talpykla su jau esamais duomenimis arba tuščia
print( 'Vietinės talpyklos aplankas:', db_folder)

# Nuskaitomas įrašų sąrašas, suformuojamas atitinkamas dataframe df_list
file_path = Path(db_path, 'list_tst.json')
with open(file_path,'r', encoding='UTF-8', errors = 'ignore') as f:
    data = json.loads(f.read())
df_list = pd.json_normalize(data, record_path =['data'])
print(df_list)

# Susirandame anotacijų pasiskirstymą per įrašus
annot_list = ['N', 'S', 'V', 'U']
# Susikuriame pagalbinį anotacijų žodyną - dictionary beats_annot
nr_sequence = list(range(len(annot_list)))
beats_annot = dict(zip(annot_list, nr_sequence))
labels_rec_all = get_annotations_distribution(df_list, db_path, beats_annot)
# print(labels_rec_all)

# Susirandame anotacijų pasiskirstymą per pacientus ir pacientų skaičių
df_sum = labels_rec_all.groupby(['userId'],sort = False).sum()

# https://sparkbyexamples.com/pandas/pandas-groupby-sum-examples/
count = labels_rec_all['userId'].value_counts()
print(f'\nViso pacientų: {len(count)}  EKG įrašų: {len(labels_rec_all)}')
print(f'\nĮrašų pasiskirstymas per pacientus')
count = count.rename("count")
frames = [df_sum, count]
result = pd.concat(frames, axis = 1)
# result.index.rename ('userID', inplace= True)
# https://www.shanelynn.ie/pandas-drop-delete-dataframe-rows-columns/

print(result)

# Susumuojamos anotacijos
# suma = labels_rec_all.iloc[:,2:7].sum()
suma = result.iloc[:,0:7].sum()
print('\n',' '*18, 'sum: ',str(suma.tolist())[1:-1])

filepath = Path(db_path, 'result.csv') 
result.to_csv(filepath)
print(f'\nFailų sąrašas ir rezultatai įrašyti į:  {filepath}')    



OS in my system :  win32

Skriptas parodo anotacijų statistiką vietinėje ZIVE talpykloje
Vietinės talpyklos aplankas: DUOM_VU
      file_name                    userId               recordingId     N    S    V  U  incl  flag               recorded_at
0   1630737.001  61632b451e325518bb017bbd  61632d471e32559827017d73   720    1    0  0     0     0  2021-09-04T06:30:01.000Z
1   1630737.002  6190d4e63cd1d227c1303cea  6190d6d13cd1d2267a303e14   557    1    0  0     0     0  2021-09-04T06:30:02.000Z
2   1630737.003  6144c588bd0cc52ba0275362  6145b1a6bd0cc5261f275908   524    0   47  0     0     0  2021-09-04T06:30:03.000Z
3   1630737.004  61a8863e5cd547e980a4e970  61a88dd3e8f900b723c6e6ec   688  115    0  0     0     0  2021-09-04T06:30:04.000Z
4   1630737.005  61632ace1e32557a62017bba  61632c1f1e32556a1c017c40   961   85    0  0     0     0  2021-09-04T06:30:05.000Z
5   1630737.006  61632ace1e32557a62017bba  61632c1f1e32553353017bef   943   70    3  0     0     0  2021-09-04T06:30:06.000Z